In [1]:
from osgeo import gdal
import numpy as np
import subprocess
import rasterio
import sys
import os
from osgeo_utils import gdal_merge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
gdal.UseExceptions()
matplotlib.use('TkAgg')

B02 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R60m\T36UUB_20190821T085601_B02_60m.jp2"
B03 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R60m\T36UUB_20190821T085601_B03_60m.jp2"
B04 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R60m\T36UUB_20190821T085601_B04_60m.jp2"
B08 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R60m\T36UUB_20190821T085601_B8A_60m.jp2"

canals = [gdal.Open(B04), gdal.Open(B03), gdal.Open(B02), gdal.Open(B08)]
cols, rows = canals[0].RasterXSize, canals[0].RasterYSize
proj = canals[0].GetProjection()
geotransform = canals[0].GetGeoTransform()

# Функція для безпечного видалення
def safe_remove(path):
    if os.path.exists(path):
        try:
            os.remove(path)
        except PermissionError:
            print(f"⚠ Не вдалося видалити {path} — файл зайнятий.")

# Створення merged_output_1_.tif
output_1 = 'merged_output_1_.tif'
safe_remove(output_1)

driver = gdal.GetDriverByName('GTiff')
out_tif = driver.Create(output_1, cols, rows, 4, gdal.GDT_UInt16)
for i, ds in enumerate(canals):
    band = ds.GetRasterBand(1).ReadAsArray()
    out_band = out_tif.GetRasterBand(i + 1)
    out_band.WriteArray(band)
    out_band.SetNoDataValue(0)
out_tif.FlushCache()
out_tif = None

# Нормалізація

def read_and_normalize(path):
    band = gdal.Open(path).ReadAsArray().astype(np.float32)
    vmin, vmax = np.percentile(band, (2, 98))
    band = np.clip(band, vmin, vmax)
    band = (band - vmin) / (vmax - vmin) * 255
    return band.astype(np.uint8)

red, green, blue, RB08 = map(read_and_normalize, [B04, B03, B02, B08])
rgb = np.dstack((red, green, blue, RB08))

plt.figure(figsize=(10, 10))
plt.imshow(rgb)
plt.axis('off')
plt.title('Composite')
plt.show()

# gdal_merge утиліти

def run_gdal_merge(output_file, input_files):
    safe_remove(output_file)
    cmd = [sys.executable, "-m", "osgeo_utils.gdal_merge", "-separate", "-of", "GTiff", "-o", output_file] + input_files
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    if result.returncode != 0:
        print("Помилка gdal_merge:\n", result.stderr)
    return result.returncode == 0

# merged_output_2.tif
output_2 = 'merged_output_2_2.tif'
run_gdal_merge(output_2, [B02, B03, B04, B08])

# merged_output_RGB.tif
output_rgb = 'merged_output_RGB_2.tif'
run_gdal_merge(output_rgb, [B02, B03, B04])

# Перепроєкція
pereproj = 'pereprojection.tif'
safe_remove(pereproj)
gdal.Warp(pereproj, output_2, dstSRS='EPSG:4326', xRes=0.0001, yRes=0.0001, format='GTiff')

# Обрізка по шейпфайлу
cutting = 'Cutting.tif'
cutline_path = r"C:\Users\Dasha Koshykova\Downloads\lab_7_data\Kyiv_regions.shp"
safe_remove(cutting)
gdal.Warp(destNameOrDestDS=cutting, srcDSOrSrcDSTab=output_2, format='GTiff',
          cutlineDSName=cutline_path, cropToCutline=True, dstNodata=0)


B02 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R60m\T36UUB_20190821T085601_B02_60m.jp2"
B03 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R60m\T36UUB_20190821T085601_B03_60m.jp2"
B04 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R60m\T36UUB_20190821T085601_B04_60m.jp2"
B08 = r"C:\Users\Dasha Koshykova\PycharmProjects\pythonProject5\lab7\S2A_MSIL2A_20190821T085601_N0213_R007_T36UUB_20190821T115206.SAFE\GRANULE\L2A_T36UUB_A021740_20190821T085815\IMG_DATA\R20m\T36UUB_20190821T085601_B8A_20m.jp2"

# Паншарпенінг
methods = ['nearest', 'bilinear', 'cubic', 'cubicspline', 'lanczos', 'average']
python_path = r"C:/Users/Dasha Koshykova/ad/python.exe"
pansharpen_script = r"C:/Users/Dasha Koshykova/ad/Scripts/gdal_pansharpen.py"

for method in methods:
    output = f"pansharpened_{method}.tif"
    safe_remove(output)
    command = [python_path, pansharpen_script, B08, B02, B03, B04, output, "-r", method, "-of", "GTiff"]
    print(f"Running with method: {method}")
    subprocess.run(command, check=True)

# Оцінка якості
original_rgb = output_rgb
ds_rgb = gdal.Open(original_rgb)
rgb_bands = [ds_rgb.GetRasterBand(i+1).ReadAsArray() for i in range(3)]
rows, cols = ds_rgb.RasterYSize, ds_rgb.RasterXSize

results = {}
for method in methods:
    pan_file = f'pansharpened_{method}.tif'
    ds_pan = gdal.Open(pan_file)
    if ds_pan is None:
        print(f"Не вдалося відкрити {pan_file}")
        continue

    pan_resized = []
    for i in range(3):
        band = ds_pan.GetRasterBand(i+1).ReadAsArray()
        factor_row = band.shape[0] // rows
        factor_col = band.shape[1] // cols
        band_small = band[::factor_row, ::factor_col][:rows, :cols]
        pan_resized.append(band_small)

    mae = [mean_absolute_error(rgb_bands[i].flatten(), pan_resized[i].flatten()) for i in range(3)]
    rmse = [np.sqrt(mean_squared_error(rgb_bands[i].flatten(), pan_resized[i].flatten())) for i in range(3)]
    r2 = [r2_score(rgb_bands[i].flatten(), pan_resized[i].flatten()) for i in range(3)]

    results[method] = {
        'MAE': np.mean(mae),
        'RMSE': np.mean(rmse),
        'R2': np.mean(r2)
    }

# Виведення результатів
print("Метрики точності паншарпенінгу:")
print(f"{'Метод':<15} {'MAE':<10} {'RMSE':<10} {'R2':<10}")
for method, metrics in results.items():
    print(f"{method:<15} {metrics['MAE']:<10.2f} {metrics['RMSE']:<10.2f} {metrics['R2']:<10.3f}")

# Графік R2
methods_sorted = list(results.keys())
r2_values = [results[m]['R2'] for m in methods_sorted]

plt.figure(figsize=(10, 5))
plt.bar(methods_sorted, r2_values, color='skyblue')
plt.ylabel("R² Score")
plt.title("Оцінка якості паншарпенінгу (R²)")
plt.grid(axis='y')
plt.show()





0...10...20...30...40...50...60...70...80...90...100 - done.

0...10...20...30...40...50...60...70...80...90...100 - done.

Running with method: nearest
Running with method: bilinear
Running with method: cubic
Running with method: cubicspline
Running with method: lanczos
Running with method: average
Метрики точності паншарпенінгу:
Метод           MAE        RMSE       R2        
nearest         5803.78    178.44     -72.319   
bilinear        5803.47    178.44     -72.019   
cubic           5803.69    178.42     -72.258   
cubicspline     5803.71    178.43     -71.915   
lanczos         5804.13    178.40     -72.388   
average         5803.78    178.44     -72.319   
